In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pygeocoder import Geocoder

# Import API key
from config import g_key

gmaps.configure(api_key = g_key)

csv_path = "../weather_py/output_data/weather_csv"

weather_data = pd.read_csv(csv_path)
weather_data.dropna()

,city,country,lat,lng,max_temp,humidity,cloudiness,wind_speed,date
0,ilhabela,br,-23.78,-45.36,70.502,74,36,1.43,2020-08-09
1,ancud,cl,-41.87,-73.82,46.400,70,75,5.70,2020-08-09
2,cape town,za,-33.93,18.42,57.992,87,40,3.10,2020-08-09
3,hermanus,za,-34.42,19.23,57.992,74,46,3.13,2020-08-09
4,qaanaaq,gl,77.48,-69.36,41.450,65,0,0.63,2020-08-09
...,...,...,...,...,...,...,...,...,...
541,tabou,ci,4.42,-7.35,75.632,79,2,2.91,2020-08-09
542,opelousas,us,30.53,-92.08,80.996,88,40,1.50,2020-08-09
543,cordoba,ar,-31.41,-64.18,84.200,26,0,6.20,2020-08-09
544,melfi,td,41.00,15.65,80.600,54,5,2.10,2020-08-09


In [2]:
location = weather_data[["lat", "lng"]].astype(float)
humidity = weather_data["humidity"].astype(float)

In [3]:
fig = gmaps.figure()

heatmap_layer = gmaps.heatmap_layer(location, weights=humidity, 
                                    dissipating=False, max_intensity=100, point_radius = 1)

fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
best_cities = weather_data[weather_data["max_temp"] <= 90]
best_cities = best_cities[best_cities["max_temp"] >= 75]
best_cities = best_cities[best_cities["humidity"] <= 80]
best_cities = best_cities[best_cities["humidity"] >= 40]
best_cities = best_cities[best_cities["cloudiness"] <= 20]
best_cities = best_cities[best_cities["wind_speed"] < 4]
best_cities

,city,country,lat,lng,max_temp,humidity,cloudiness,wind_speed,date
51,broome,au,42.25,-75.83,84.992,44,1,2.10,2020-08-09
57,san vicente,ph,-26.62,-54.13,75.632,41,0,1.03,2020-08-09
86,eureka,us,40.80,-124.16,75.992,77,1,2.10,2020-08-09
110,portland,au,45.52,-122.68,75.002,52,1,1.50,2020-08-09
126,savalou,bj,7.93,1.98,75.596,76,5,0.68,2020-08-09
135,candido de abreu,br,-24.57,-51.33,75.794,42,0,0.21,2020-08-09
194,sao gabriel da cachoeira,br,-0.13,-67.09,87.800,66,20,1.50,2020-08-09
223,fortuna,us,40.60,-124.16,75.992,62,1,1.34,2020-08-09
331,la palma,pa,33.85,-118.05,87.008,51,1,3.86,2020-08-09
345,caraballeda,ve,10.61,-66.85,82.004,56,16,3.80,2020-08-09


In [11]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

search_type = "hotel"
radius = 5000
hotel_names = []
hotel_lats = []
hotel_lngs = []

params = {
   
    "radius": radius,
    "type": search_type,
    "key": g_key
}

for index, row in best_cities.iterrows():
    
    try:
       
        lat = row["lat"]
        lng = row["lng"]
        city = row["city"]
        
        params["location"] = f"{lat},{lng}"

        search = requests.get(base_url, params=params)
        search_result = search.json()
        
        hotel_name = search_result["results"][0]["name"]
        hotel_lat = search_result["results"][0]["geometry"]["location"]["lat"] 
        hotel_lng = search_result["results"][0]["geometry"]["location"]["lng"] 
        
        hotel_names.append(hotel_name)
        hotel_lats.append(hotel_lat) 
        hotel_lngs.append(hotel_lng)
        
        print(f"found {hotel_name} in {city}")
              
    except:
              
        print("oops. no hotel found") 

found Chenango Forks in broome
found San Pedro in san vicente
found Eureka in eureka
found Portland in portland
found Savalou in savalou
found Cândido de Abreu in candido de abreu
found São Gabriel da Cachoeira in sao gabriel da cachoeira
found Fortuna in fortuna
found Long Beach in la palma
found Caraballeda in caraballeda
found Ismailia in ismailia
found Alcúdia in alcudia
found Mitsamiouli in mitsamiouli
found Tabou in tabou
found Melfi in melfi


In [15]:
best_cities["hotel"] = hotel_names
best_cities["hotel_lat"] = hotel_lats
best_cities["hotel_lng"] = hotel_lngs
best_cities

,city,country,lat,lng,max_temp,humidity,cloudiness,wind_speed,date,hotel,hotel_lat,hotel_lng
51,broome,au,42.25,-75.83,84.992,44,1,2.10,2020-08-09,Chenango Forks,42.238018,-75.846302
57,san vicente,ph,-26.62,-54.13,75.632,41,0,1.03,2020-08-09,San Pedro,-26.621583,-54.109311
86,eureka,us,40.80,-124.16,75.992,77,1,2.10,2020-08-09,Eureka,40.802071,-124.163673
110,portland,au,45.52,-122.68,75.002,52,1,1.50,2020-08-09,Portland,45.505106,-122.675026
126,savalou,bj,7.93,1.98,75.596,76,5,0.68,2020-08-09,Savalou,7.930137,1.971028
135,candido de abreu,br,-24.57,-51.33,75.794,42,0,0.21,2020-08-09,Cândido de Abreu,-24.565377,-51.337678
194,sao gabriel da cachoeira,br,-0.13,-67.09,87.800,66,20,1.50,2020-08-09,São Gabriel da Cachoeira,-0.115566,-67.088305
223,fortuna,us,40.60,-124.16,75.992,62,1,1.34,2020-08-09,Fortuna,40.598187,-124.157276
331,la palma,pa,33.85,-118.05,87.008,51,1,3.86,2020-08-09,Long Beach,33.770050,-118.193740
345,caraballeda,ve,10.61,-66.85,82.004,56,16,3.80,2020-08-09,Caraballeda,10.613540,-66.849888


In [16]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{hotel}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in best_cities.iterrows()]
locations = best_cities[["hotel_lat", "hotel_lng"]]

markers = gmaps.marker_layer(locations)

fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))